#### This code gets all the monthly data from yahoo finance for a list of tickers.  We chose a monthly timeframe because a lot of the data we want to compare it is also in a monthly format.

In [1]:
    import datetime
    import time
    import urllib.request
    import pandas as pd

#### The following function retrieves all historical data in yahoo finance back to 1/1/1980 for a particular ticker.  It returns a dataframe for one ticker.  If data doesn't go back as far as 1980 for the ticker, data will be returned as far back as possible.  

In [8]:
def get_df_from_yahoo_finance (symbol): 

    end_date = datetime.datetime.now()
    end_date = int(round(end_date.timestamp())) * 1000  #<-- Convert datetime to milliseconds 
    end_date_str = str(end_date)
    end_date_str = end_date_str[:10]  #<-- This removes all but the last 10 millisecond values for the time which is format yahoo wants

    start_date_str = '345427200'
  

    ticker_dataframe = {}
    

    # Build Yahoo finance URL for the data request.
    
    yahoo_finance_url_with_symbol = ('https://query1.finance.yahoo.com/v7/finance/download/' + str(symbol)
                                     + '?period1=' + str(start_date_str) + '&period2=' + str(end_date_str)
                                     + '&interval=1mo&events=history&includeAdjustedClose=true')

    url2 = urllib.request.urlopen(yahoo_finance_url_with_symbol) #request data (comes as CSV)
    
    ticker_dataframe = pd.read_csv(url2)  #convert CSV in memory from the Yahoo request to dataframe
    ticker_dataframe.insert(1, "Ticker", symbol)

    return (ticker_dataframe)


#### This next section iterates through the list of tickers and appends them all to one dataframe as rows.  Add additonal tickers to the list_of_tickers_for_yahoo_data_dump list object.  

In [23]:

list_of_tickers_for_yahoo_data_dump = ['SPY', 'GLD', 'XLE', 'XLV', 'XLP', 'QQQ', 'AMT', 'AMC', 'VNQ', 'VPU', 'BTC-USD', 'BND', '^VIX', 'JPM']   # <-- Add tickers here

tickers_and_data_df = pd.DataFrame({})

for ticker in list_of_tickers_for_yahoo_data_dump:
    
    single_ticker_df = get_df_from_yahoo_finance(ticker)
    tickers_and_data_df = pd.concat([tickers_and_data_df, single_ticker_df])


#### Print the new dataframe --> tickers_and_data_df.

In [24]:
print (tickers_and_data_df)   #View the dataframe we just created

           Date Ticker        Open        High         Low       Close  \
0    1993-02-01    SPY   43.968750   45.125000   42.812500   44.406250   
1    1993-03-01    SPY   44.562500   45.843750   44.218750   45.187500   
2    1993-04-01    SPY   45.250000   45.250000   43.281250   44.031250   
3    1993-05-01    SPY   44.093750   45.656250   43.843750   45.218750   
4    1993-06-01    SPY   45.375000   45.812500   44.218750   45.062500   
..          ...    ...         ...         ...         ...         ...   
448  2022-05-01    JPM  119.879997  133.149994  115.019997  132.229996   
449  2022-06-01    JPM  132.869995  132.869995  110.930000  112.610001   
450  2022-07-01    JPM  112.650002  116.500000  106.059998  115.360001   
451  2022-08-01    JPM  114.500000  115.110001  111.019997  112.360001   
452  2022-08-05    JPM  112.529999  116.189903  112.050003  115.760002   

      Adj Close     Volume  
0     25.709496    5417600  
1     26.161802    3019200  
2     25.612785    26972

#### To view a dataframe of a single ticker use the following

In [25]:
btc_df = tickers_and_data_df.loc[tickers_and_data_df['Ticker'] == 'BTC-USD']
print(btc_df)

          Date   Ticker          Open          High           Low  \
0   2014-10-01  BTC-USD    387.427002    411.697998    289.295990   
1   2014-11-01  BTC-USD    338.649994    457.092987    320.626007   
2   2014-12-01  BTC-USD    378.248993    384.037994    304.231995   
3   2015-01-01  BTC-USD    320.434998    320.434998    171.509995   
4   2015-02-01  BTC-USD    216.867004    265.610992    212.014999   
..         ...      ...           ...           ...           ...   
91  2022-05-01  BTC-USD  37713.265625  39902.949219  26350.490234   
92  2022-06-01  BTC-USD  31792.554688  31957.285156  17708.623047   
93  2022-07-01  BTC-USD  19820.470703  24572.580078  18966.951172   
94  2022-08-01  BTC-USD  23336.718750  23578.650391  22485.701172   
95  2022-08-06  BTC-USD  23298.335938  23298.335938  23235.458984   

           Close     Adj Close         Volume  
0     338.321014    338.321014      902994450  
1     378.046997    378.046997      659733360  
2     320.192993    320.192

In [29]:
pivot = pd.pivot_table(
    data=tickers_and_data_df,
    index='Date',
    columns ='Ticker',
    
)

pivot = pivot.dropna()


In [28]:
# from pathlib import Path  
# filepath = Path('data/pivot.csv')  
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# pivot.to_csv(filepath)  

In [30]:
pivot = pivot['Close']

In [31]:
pivot


Ticker,AMC,AMT,BND,BTC-USD,GLD,JPM,QQQ,SPY,VNQ,VPU,XLE,XLP,XLV,^VIX
Date,,,,,,,,,,,,,,
2014-10-01,25.400000,97.500000,82.360001,338.321014,112.660004,60.480000,101.400002,201.660004,78.989998,98.959999,87.419998,46.709999,67.269997,14.030000
2014-11-01,26.219999,105.010002,82.870003,378.046997,112.110001,60.160000,106.010002,207.199997,80.570000,99.760002,79.820000,49.299999,69.610001,13.330000
2014-12-01,26.180000,98.849998,82.370003,320.192993,113.580002,62.580002,103.250000,205.539993,81.000000,102.349998,79.160004,48.490002,68.379997,19.200001
2015-01-01,28.120001,96.949997,84.349998,217.464005,123.449997,54.380001,101.099998,199.449997,86.550003,104.580002,75.550003,48.020000,69.269997,20.969999
2015-02-01,34.380001,99.139999,83.080002,254.263000,116.160004,61.279999,108.400002,210.660004,83.370003,98.370003,79.019997,50.009998,72.239998,13.340000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-01,15.300000,241.020004,76.190002,37714.875000,176.910004,119.360001,313.250000,412.000000,103.940002,154.729996,75.150002,77.639999,130.289993,33.400002
2022-05-01,14.340000,256.130005,76.680000,31792.310547,171.139999,132.229996,308.279999,412.929993,99.070000,161.710007,87.199997,74.470001,132.229996,26.190001
2022-06-01,13.550000,255.589996,75.260002,19784.726563,168.460007,112.610001,280.279999,377.250000,91.110001,152.350006,71.510002,72.180000,128.240005,28.709999
